# OpenUVF Image Segmentation

Copyright © 2019 Southern Company Services, Inc.  All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
#Import Statements
import cv2 as cv
import os
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import math
import lensfunpy
import tensorflow as tf
import core.utils.segmentation_utils as segment
import core.utils.detection_utils as detect
from random import sample
from PIL import Image as im
from core import object_detection as object_detection
from core.object_detection.utils import ops as utils_ops
from core.object_detection.utils import label_map_util
from core.object_detection.utils import visualization_utils as vis_util


C:\Users\bhg19\OneDrive\Documents\GitHub\OpenUVF\core\object_detection\utils\visualization_utils.py:27: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements


### Inputs

#### Directories
1. images_dir = string specifiying the path (local or global) to the image directory


#### Module parameters
1. module_orientation = string specifying the orientation of the module. Choices are 'vertical' or ' horizontal' 
2. module_aspect_ratio = float specifying the aspect ratio of the module, specified as ratio of longer side to shorter side


#### Function Settings
1. debug = boolean specifying if debug plots should be generated
2. itmax = int specifying the maximum amount of refinement iterations where the segmentation routine is adjusted to increase accuracy

In [2]:
# Directories
images_dir = 'workspace\module_segmentation\Sample Set\FullSize'
images_ext = '.JPG'
output_dir = 'workspace\module_segmentation\Sample Set\Processed/'

# Imaging Parameters 
num_rows = 1
module_orientation = 'horizontal'
module_aspect_ratio = 2
camera_orientation = 'landscape'

# Camera Distortion Parameters
correct_distortion = True
correct_distortion_mode = 'lensfun'
camera_mnfcr = 'NIKON CORPORATION'
camera_model = 'NIKON D3S'
lens_mnfcr = 'SAMYANG'
lens_model = 'Samyang T-S 24mm f/3.5 ED AS UMC'
image_focal_length = 24.0
image_aperture = 1.4
approx_distance = 3
camera_calibration = None

# Module Detection Parameters
path_to_frozen_graph = 'workspace/module_segmentation/models/FRCNN_RN101_v2/frozen_inference_graph.pb'
path_to_labels = 'workspace/module_segmentation/1_Class_label_map.pbtxt'



# Function Settings
debug = True
debug_outputs = True
itmax = 10

### Default Plotting Parameters

In [3]:
mpl.rcParams.keys()
mpl.rcParams['axes.spines.bottom'] = False
mpl.rcParams['axes.spines.left'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['xtick.bottom'] = False
mpl.rcParams['ytick.left'] = False

## Define Output Directories

In [4]:
prelim_output_dir = os.path.join(output_dir, 'prelim_perspective_correction')
detect_output_dir = os.path.join(output_dir, 'module_detection')
final_output_dir = os.path.join(output_dir, 'final')
prelim_debug_output_dir = os.path.join(output_dir, 'prelim_debug')
module_debug_output_dir = os.path.join(output_dir, 'module_debug')

### Debug Plots

In [5]:
def generateDebugPlots():
        
    #Create figure
    fig = plt.figure(num=f'Image {image_name} Preprocessing', tight_layout=True,figsize=[18, 9.5])
    gs = gridspec.GridSpec(4,5)
    mng = plt.get_current_fig_manager()

    # Axis RGB - Plot base image
    axrgb = fig.add_subplot(gs[0,0])
    plt.imshow(rgb)
    plt.title('RGB')

    # Axis R - Plot base image
    axr = fig.add_subplot(gs[0,1])
    plt.imshow(red) 
    plt.title('Red')

    # Axis G - Plot base image
    axg = fig.add_subplot(gs[0,2])
    plt.imshow(gre) 
    plt.title('Green')

    # Axis B - Plot base image
    axb = fig.add_subplot(gs[0,3])
    plt.imshow(blu) 
    plt.title('Blue')

    # Axis RGB Hist - Plot histograms of the three channels
    axrgbhist = fig.add_subplot(gs[0,4])
    plt.plot(red_hist, 'r-'), plt.plot(gre_hist, 'g-'), plt.plot(blu_hist, 'b-')
    plt.xlabel('Intensity'), plt.ylabel('Count'), plt.title('RGB Histograms')

    # Axis HSV - Plot image in hsv color space
    axhsv = fig.add_subplot(gs[1,0])
    plt.imshow(hsv)
    plt.title('HSV')

    # Axis H - Plot image in hsv color space
    axh = fig.add_subplot(gs[1,1])
    plt.imshow(hue)
    plt.title('Hue')

    # Axis S - Plot image in hsv color space
    axs = fig.add_subplot(gs[1,2])
    plt.imshow(sat)
    plt.title('Saturation')

    # Axis V - Plot image in hsv color space
    axv = fig.add_subplot(gs[1,3])
    plt.imshow(val)
    plt.title('Value (brightness)')

    # Axis Hist - Plot histograms of the three channels
    axhsvhist = fig.add_subplot(gs[1,4])
    plt.plot(hue_hist, 'k-', label = 'Hue')
    plt.plot(sat_hist, 'm-', label = 'Sat')
    plt.plot(val_hist, 'c-', label = 'Val')
    plt.xlabel('Intensity'), plt.ylabel('Count'), plt.title('HSV Histograms'), plt.legend(loc='upper left')

    # Axis Grau - Plot standard grayscale image
    axgray = fig.add_subplot(gs[2,0])
    plt.imshow(gray)
    plt.title('Grayscale')

    # Axis Gray Histogram
    axgrayhist = fig.add_subplot(gs[2,1])
    gray_hist = cv.calcHist([gray], [0], None, [256], [0, 256])
    plt.plot(gray_hist, 'k-')
    plt.title('Grayscale Histogram')

    # Axis RGB min - Plot standard grayscale image
    axgray = fig.add_subplot(gs[2,2])
    plt.imshow(rgb_min)
    plt.title('RGB Min (min of each channel)')

    # Axis RGB min Histogram
    axgrayhist = fig.add_subplot(gs[2,3])
    rgb_min_hist = cv.calcHist([rgb_min], [0], None, [256], [0, 256])
    plt.plot(rgb_min_hist, 'k-')
    plt.title('RGB min Histogram')

    # Axis RGB min edges 
    axhe = fig.add_subplot(gs[3,0])
    plt.imshow(edges_rgb_min)
    plt.title('RGB Min Edges')

    # Axis Gray edges - Plot showing the edges detected in the standard grayscale image
    axhe = fig.add_subplot(gs[3,1])
    plt.imshow(edges_gray)
    plt.title('Grayscale Edges')

    # Axis Hue edges - Plot showing the edge
    axhe = fig.add_subplot(gs[3,2])
    plt.imshow(edges_hue)
    plt.title('Hue Edges')

    # Axis final edges - Plot showing the edge
    axhe = fig.add_subplot(gs[3,3])
    plt.imshow(edges)
    plt.title('Edges')

    # Axis Hue Lines - Plot detected lines 
    axhe = fig.add_subplot(gs[3,4])
    plt.imshow(gray_bgr)
    plt.title('Detected Lines (Hough)')

    #Create Second figure
    fig2 = plt.figure(num=f'Image {image_name} Perspective Correction', tight_layout=True,figsize=[18, 9.5])
    gs2 = gridspec.GridSpec(2,3)
    mng2 = plt.get_current_fig_manager()

    

    #Vanishing point plot
    if len(vert_vanish_pt) >= 2:
        axvps = fig2.add_subplot(gs2[0,1])
        plt.imshow(gray_bgr)
        plt.plot(vert_vanish_xs, vert_vanish_ys, 'b.')
        plt.plot(vert_vanish_pt[0], vert_vanish_pt[1], 'r.', markersize=16)  
        plt.plot(hori_vanish_xs, hori_vanish_ys, 'g.')
        plt.xlim(-10000, 10000), plt.ylim(-10000, 10000)
        plt.title('Vanishing Points')

    #Projective transform plots
    if sufficient_lines:
    
        #Selected representative perspective lines
        axls = fig2.add_subplot(gs2[0,0])
        plt.imshow(gray_sample_lines)
        plt.title('Projective Transform Sample Lines')
    
        #Projective Transform Geometry Calculations Plot
        axptg = fig2.add_subplot(gs2[0,2])
        plt.imshow(gray_sample_box)
        plt.title('Projective Transform Geometry')

        #Projective Transform 
        axptg = fig2.add_subplot(gs2[1,0])
        plt.imshow(transed)
        plt.title('Projective Transform')

    #Update figure and Pause
    plt.draw()
    fig.canvas.manager.window.raise_()
    plt.pause(0.01)
    
    #Save all images as files
    if debug_outputs:
        
        # Sample lines 
        output_path = os.path.join(prelim_debug_output_dir, image_root + '_lines' + images_ext)
        cv.imwrite(output_path, cv.cvtColor(gray_sample_lines, cv.COLOR_BGR2RGB))
        
        # Sample Boxes 
        output_path = os.path.join(prelim_debug_output_dir, image_root + '_boxes' + images_ext)
        cv.imwrite(output_path, cv.cvtColor(gray_sample_box, cv.COLOR_BGR2RGB))
        
        # Debug figures 
        output_path = os.path.join(prelim_debug_output_dir, image_root + '_debug_1' + images_ext)
        fig.savefig(output_path)
        output_path = os.path.join(prelim_debug_output_dir, image_root + '_debug_2' + images_ext)
        fig2.savefig(output_path)
        
        #Close figures
        plt.close('all')

## Setup

In [6]:
%matplotlib 
# Define list of images in the specified directory (NEEDS EDGE HANDLING)
image_list = os.listdir(images_dir)
image_list = [image_name for image_name in image_list if image_name.endswith(images_ext)]

# Predefined global image processing parameters
resize_rows = 600 
resize_cols = 902            #only used when running module detection alone

Using matplotlib backend: Qt5Agg


## Preliminary Perspective Correction

In [7]:


#Processing steps
equalize_hist = False
equalize_method = 'global'
    
#iterate through images
image_it = 1
for image_name in image_list:
    
    #Log start time
    time_start = time.time()
    
    #Debug Outputs
    print(f'\nProcessing Image {image_name} ({image_it}):') 
    
    #Define image path
    image_root = os.path.splitext(image_name)[0]
    image_path = images_dir + '/' + image_name
    
    #Load image
    image_original = cv.imread(image_path)
    
    #Calculate basic parameters
    image_rows, image_cols, image_channels = image_original.shape;
    aspect_ratio = image_cols/image_rows
    image_pixels = image_cols * image_rows
    image_scalar = resize_rows/image_rows
    resize_cols = round(aspect_ratio * resize_rows)
    resize_pixels = resize_rows * resize_cols
    print(f'   Image Loading:')
    print(f'      Aspect Ratio: {aspect_ratio:.2f}')
    print(f'      Image Size is {image_rows} x {image_cols} ({image_pixels} px.)')
    print(f'      Image Resized to {resize_rows} x {resize_cols} ({resize_pixels} px.)')
    
    #Distortion correction - POSSIBLY IMPLEMENT - May cause more distortion than it corrects
    if correct_distortion:
        
        #Print update statement
        print('   Camera Distortion Correction:')
        
        if correct_distortion_mode == 'lensfun':                 #Corrects using lensfunpy, which uses the lensfun database
            
            #Error Handling
            try: 
                
                #Load camera and lens information
                lensfun_db = lensfunpy.Database()
                camera = lensfun_db.find_cameras(camera_mnfcr, camera_model)[0]
                lens = lensfun_db.find_lenses(camera, lens_mnfcr, lens_model)[0]

                #Correct distortion
                distortion_modifier = lensfunpy.Modifier(lens, camera.crop_factor, image_cols, image_rows)
                distortion_modifier.initialize(image_focal_length, image_aperture, approx_distance)
                undistorted_coords = distortion_modifier.apply_geometry_distortion()
                undistorted = cv.remap(image_original, undistorted_coords, None, cv.INTER_LANCZOS4)
                
                #Print update
                print(f'      Correction applied for {camera_model} with {lens_model}.')
                
            except Exception:
                
                print(Exception)
                
                #Print update
                print(f'      Correction failed. Invalid camera or lens inputs.')
            
        elif correct_distortion_mode == 'camera_calibration':     #Uses an existing calibration file from calibrate_camera utility
            
            x=y
            
        else:
            print('      Distortion Correction Failed. Check inputs')
            undistorted = image_original
    
    #Resize image to nominal size to increase computation speed(and adjust the anticipated aspect ratio as well)
    
    image = cv.resize(undistorted, (resize_cols, resize_rows))
        
    #NEED PREPROCESSING STEP WHICH ADJUSTS THE WHITE BALANCE SOMEHOW - WHEN IMAGING, WB should be constant
    
    #Convert to RGB and isolate channels
    rgb = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    rgb_min = np.amin(rgb, axis=2)
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    red = rgb[:,:,0]
    gre = rgb[:,:,1]
    blu = rgb[:,:,2]
    
    #Calculate RGB statistics from the image 
    red_mean = np.mean(red)
    red_hist = cv.calcHist([rgb], [0], None, [256], [0, 256])
    gre_mean = np.mean(gre)
    gre_hist = cv.calcHist([gre], [0], None, [256], [0, 256])
    blu_mean = np.mean(blu)
    blu_hist = cv.calcHist([blu], [0], None, [256], [0, 256])
    print(f'   Image Statistics:')
    print(f'      RGB Means: red = {red_mean:.2f};'\
          f' blue = {blu_mean:.2f}; green = {gre_mean:.2f}')
    
    
    #Convert to HSV and isolate channels
    hsv = cv.cvtColor(rgb, cv.COLOR_RGB2HSV)
    hue = hsv[:,:,0]
    sat = hsv[:,:,1]
    val = hsv[:,:,2]
    
    #Calculate HSV statistics from the image
    #hue = hue.astype(np.float64)
    #hue[hue < 48] = hue + 48
    hue_mean = np.mean(hue)
    hue_hist = cv.calcHist([hue], [0], None, [256], [0, 256])
    sat_mean = np.mean(sat)
    sat_hist = cv.calcHist([sat], [0], None, [256], [0, 256])
    val_mean = np.mean(val)
    val_hist = cv.calcHist([val], [0], None, [256], [0, 256])
    print(f'      HSV Means: hue = {hue_mean:.2f};'\
          f' saturation = {sat_mean:.2f}; value = {val_mean:.2f}')

    #Detect if image is particularly dark 
    
    
    
    #Histogram Equalization
    if equalize_hist and (equalize_method == 'adaptive'):
        clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
        rgb_min = clahe.apply(rgb_min)
        gray = clahe.apply(gray)
        hue = clahe.apply(hue)
    elif equalize_hist and (equalize_method == 'global'):
        rgb_min = cv.equalizeHist(rgb_min)
        gray = cv.equalizeHist(gray)
        hue = cv.equalizeHist(hue)
    
    
    #Iteratively refine detected edges in the images
    edges, edge_outputs = segment.edgeDetection(rgb, rgb_min, gray, hue, image_pixels, debug=debug)
    rgb_min_smooth = edge_outputs['rgb_min_smooth']
    gray_smooth = edge_outputs['gray_smooth']
    edges_rgb_min = edge_outputs['edges_rgb_min']
    edges_gray = edge_outputs['edges_gray']
    edges_hue = edge_outputs['edges_hue']
    
    #Detect prominent lines
    print('   Line Detection:')
    max_std = 1                                     #STD cutoff for line deviation
    min_line_length = 150
    max_line_gap = 10
    hough_threshold = 100
    hough_theta = np.pi/1080
    hough_rho = 1
    line_it = 0
    line_repeat = True
    sufficient_lines = False
    while line_repeat & (line_it < 10):
    
        #Calculate lines with specified parameters
        lines, line_points, line_points_edge, line_angles = segment.lineDetection(edges, hough_rho, hough_theta, \
                                                                                  hough_threshold, min_line_length,\
                                                                                  max_line_gap, resize_rows, resize_cols, \
                                                                                  debug=debug)
        
        #Filter lines 
        vert_inds, vert_lines, vert_angles, hori_inds, hori_lines, hori_angles, reject_inds, reject_lines, line_outputs \
            = segment.filterLines(lines, line_points, line_angles, hough_theta, debug = True)
        
        #Extract additional outputs
        vert_angles_mean = line_outputs['vert_angles_mean']
        vert_angles_std = line_outputs['vert_angles_std']
        hori_angles_mean = line_outputs['hori_angles_mean']
        hori_angles_std = line_outputs['hori_angles_std']
        
        #Determine vanishing points
        vert_vanish_pt = line_outputs['vert_vanish_pt']
        vert_vanish_xs = line_outputs['vert_vanish_xs']   
        vert_vanish_ys = line_outputs['vert_vanish_ys']
        hori_vanish_xs = []
        hori_vanish_ys = []
        
        #Count number of final lines
        num_lines = len(lines)
        num_hori_lines = len(hori_lines)
        num_vert_lines = len(vert_lines)
        num_lines_filtered = num_lines -(num_hori_lines + num_vert_lines)
        
        #Determine if the line detection procedure should be repeated and strengthen or weaken parameters to achieve this
        max_ct = 250
        min_ct = 2      
        if (num_vert_lines < min_ct) or (num_hori_lines < min_ct):
            min_line_length *= 0.8 
            max_line_gap *= 1.2 
            max_std *= 1.1
        elif (num_vert_lines > max_ct) or (num_hori_lines > max_ct):
            min_line_length = 1.2 * min_line_length
            max_line_gap = 0.8 * max_line_gap
            max_std *= 0.9
        else:
            line_repeat = False
            sufficient_lines = True
            
        #Output update
        print(f'      ({line_it}): {num_lines} Lines Detected - {num_lines_filtered} Filtered - '\
              f'{num_hori_lines} Horizontal ({hori_angles_mean:.2f} +- {hori_angles_std:.2f} deg.)',\
              f'- {num_vert_lines} Vertical ({vert_angles_mean:.2f} +- {vert_angles_std:.2f} deg.)')
        
        #Iterate iteration counter
        line_it = line_it + 1
        
        #Failure update
        if line_repeat and (line_it == 10):
            print(f'      ({line_it}): Line Detection Failed - Image Rejected')
        

    #Plot lines on the image
    if debug:
        gray_bgr = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        gray_bgr = segment.plotLines(gray_bgr, vert_lines, hori_lines, reject_lines)
        gray_bgr_extended = cv.cvtColor(gray_smooth, cv.COLOR_GRAY2BGR)
        gray_bgr_extended = segment.plotLines(gray, None, None, None, all_lines = line_points_edge)
    
    
    #Plot vanishing points and perspective lines
    #if debug:
    #    gray_vanish_pts = gray_bgr
    #    for v in range(0,len(vert_vanish_xs)):
    #        if not (math.isnan(vert_vanish_xs[v])) and not (math.isnan(vert_vanish_ys[v])):
    #            cv.circle(gray_vanish_pts, (int(vert_vanish_xs[v]), int(vert_vanish_ys[v])), 1, (0,128,255), thickness=-1)
    #    for h in range(0,len(hori_vanish_xs)):
    #        if not (math.isnan(hori_vanish_xs[h])) and not (math.isnan(hori_vanish_ys[h])):
    #            cv.circle(gray_vanish_pts, (int(hori_vanish_xs[h]), int(hori_vanish_ys[h])), 1, (0,255,128), thickness=-1)
            
    #Select 4 representative lines - 2 Horizontal and 2 Vertical
    #vert_lines_sort = np.argsort(vert_angles)
    #vert_lines = vert_lines[vert_lines_sort]
    #hori_lines_sort = np.argsort(hori_angles)
    #hori_lines = hori_lines[hori_lines_sort]

    #PASTE HERE
    
    #Calculate mean projective transform
    if sufficient_lines:
        print('   Projective Transform:')
        transed, proj_trans, rot_ang, gray_sample_lines, gray_sample_box = segment.projectiveTransform(image_original, lines,\
                                                                                                   vert_inds, hori_inds,\
                                                                                                   gray_smooth, resize_rows,\
                                                                                                   image_scalar, hough_theta)

        #Save image
        transed = cv.cvtColor(transed, cv.COLOR_BGR2RGB)
        output_path = os.path.join(prelim_output_dir, image_root + '_output' + images_ext)
        try:
            cv.imwrite(output_path, cv.cvtColor(transed, cv.COLOR_BGR2RGB))
        except OSError:
            os.mkdir(prelim_output_dir)
            cv.imwrite(output_path, cv.cvtColor(transed, cv.COLOR_BGR2RGB))
    
    
    
    #Log end time and calculate total execution time
    time_end = time.time()
    time_execute = time_end - time_start
    print(f'   Processing time: {time_execute:0.4f} s')
    
    #Generate debug plots
    if debug:
        generateDebugPlots()
        
    #Iteration counter    
    image_it += 1
        
        
        
        
        

    


Processing Image DSC03003.JPG (1):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 123.14; blue = 104.39; green = 74.86
      HSV Means: hue = 139.96; saturation = 147.06; value = 137.81
   Edge Detection:
      (0): Ratio = 0.59%
   Line Detection:


C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Anaconda3\envs\tensorflowGPU\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


      (0): 80 Lines Detected - 76 Filtered - 2 Horizontal (2.21 +- 1.87 deg.) - 2 Vertical (96.16 +- 4.70 deg.)
   Projective Transform:
   Processing time: 1.6378 s

Processing Image DSC03013.JPG (2):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
   Camera Distortion Correction:
      Correction applied for NIKON D3S with Samyang T-S 24mm f/3.5 ED AS UMC.
   Image Statistics:
      RGB Means: red = 97.06; blue = 121.51; green = 60.91
      HSV Means: hue = 139.00; saturation = 149.42; value = 128.87
   Edge Detection:
      (0): Ratio = 0.52%
   Line Detection:
      (0): 113 Lines Detected - 109 Filtered - 2 Horizontal (0.65 +- 0.68 deg.) - 2 Vertical (93.41 +- 2.91 deg.)
   Projective Transform:
   Processing time: 1.5612 s

Processing Image DSC03021.JPG (3):
   Image Loading:
      Aspect Ratio: 1.50
      Image Size is 2832 x 4240 (12007680 px.)
      Image Resized to 600 x 898 (538800 px.)
 

KeyboardInterrupt: 


KeyboardInterrupt



## Feature Based Module Perspective Refinement

In [10]:
def correctModulePerspective(module_rgb, module_root, debug_dir=None, debug=True):
    
    #Create debug figure 
    if debug:
        module_fig = plt.figure(num=f'Module Perspective Correction Debug', tight_layout=True,figsize=[18, 9.5])
        gs = gridspec.GridSpec(2,3)
        mng = plt.get_current_fig_manager()
   
    


    #Alternative color space representations
    module_bgr = cv.cvtColor(module_rgb, cv.COLOR_RGB2BGR)
    module_gray = cv.cvtColor(module_bgr, cv.COLOR_BGR2GRAY)
    module_rgb_min = np.amin(module_rgb, axis=2)
    module_hsv = cv.cvtColor(module_bgr, cv.COLOR_BGR2HSV)
    module_hue = module_hsv[:,:,0]
    if debug:
        axo = module_fig.add_subplot(gs[0,0])
        plt.imshow(module_gray)
        plt.title('Module Grayscale')
        output_name = module_root + '_crop_initial' + images_ext
        output_path = os.path.join(debug_dir, output_name)
        cv.imwrite(output_path, module_rgb)
    
    #Calculate image dimensions
    module_rows, module_cols, module_channels = module_bgr.shape
    module_pixels = module_rows * module_cols
    module_min_dim = np.min([module_rows, module_cols])
    print(f'      Image Size is {module_rows} x {module_cols} ({module_pixels} px.)')
    
    #Detect Frame Outline
    #detectFrameOutline(module_rgb)
    
    
    
    #Define edge detection parameters
    edge_params = dict()
    edge_params['gauss_size'] = (49,49)
    edge_params['gauss_std'] = 3
    edge_params['canny_thresh_min'] = 150
    edge_params['canny_thresh_max'] = 220
    edge_params['edge_ratio_goal'] = 0.1
    edge_params['edge_ratio_range'] = (0.01, 0.20)
    
    #Define line detection parameters
    max_std = 1                                     #STD cutoff for line deviation
    min_line_length = int(0.5 * module_min_dim)
    max_line_gap = int(0.33 * min_line_length)
    hough_threshold = int(0.5 * min_line_length)
    hough_theta = np.pi/1080
    hough_rho = 1
    line_it = 0
    
    
    
    #Detect lines in the original image
    line_it = 0
    line_repeat = True
    print(f'\n      Line Detection/Selection in Original Image:\n')
    while line_repeat & (line_it < 10):
        
        #Print update
        #print(f'\n      Iteration {line_it}:')
        
        #Detect edges
        module_gray = cv.cvtColor(module_rgb.copy(), cv.COLOR_RGB2GRAY)
        module_edges, module_edge_outputs = segment.edgeDetection(module_rgb.copy(), module_rgb_min, module_gray.copy(), module_hue, module_pixels,remove_total_black=True, params=edge_params, debug=False)
        module_edge_its = module_edge_outputs['iterations']
        module_edge_percent = module_edge_outputs['edge_percent']
        module_thresh_low = module_edge_outputs['thresh_low']
        module_thresh_high = module_edge_outputs['thresh_high']
        module_total_black = module_edge_outputs['total_black']
        print(f'        ({line_it}): Edge detection: {module_edge_its} iterations - Ratio = {module_edge_percent:.2f}% - Thresholds = '\
              f'({module_thresh_low}, {module_thresh_high})')
        
        #Detect lines
        lines, line_points, line_points_edge, line_angles = segment.lineDetection(module_edges, hough_rho, hough_theta, \
                                                                                  hough_threshold, min_line_length,\
                                                                                  max_line_gap, module_rows, module_cols, \
                                                                                  debug=debug)
        #Filter lines 
        vert_inds, vert_lines, vert_angles, hori_inds, hori_lines, hori_angles, reject_inds, reject_lines, line_outputs \
            = segment.filterLines(lines, line_points, line_angles, hough_theta, vert_select_mode='histogram', hori_select_ct=4,\
                                  vert_select_ct=4, debug = True)
        
        #Pull line statistics
        vert_lines_stats = {'angles': line_outputs['vert_angles_all'],'angles_mean': line_outputs['vert_angles_mean'], \
                            'angles_std': line_outputs['vert_angles_std']}
        hori_lines_stats = {'angles': line_outputs['hori_angles_all'],'angles_mean': line_outputs['hori_angles_mean'], \
                            'angles_std': line_outputs['hori_angles_std']}
        
        #Pull mean angles and stds
        vert_angles_mean = vert_lines_stats['angles_mean']
        vert_angles_std = vert_lines_stats['angles_std']
        hori_angles_mean = hori_lines_stats['angles_mean']
        hori_angles_std = hori_lines_stats['angles_std']  
    
        #Count number of final lines
        num_lines = len(lines)
        num_hori_lines = len(hori_lines_stats['angles'])
        num_vert_lines = len(vert_lines_stats['angles'])
        num_hori_lines_chosen = len(hori_lines)
        num_vert_lines_chosen = len(vert_lines)
        num_lines_filtered = num_lines - (num_hori_lines + num_vert_lines)
        
        #Print update about all lines detected (not exclusively filtered lines)
        print(f'        ({line_it}): Line Detection: {num_lines} Detected - '\
              f'{num_hori_lines} Horizontal ({hori_angles_mean:.2f} +- {hori_angles_std:.2f} deg.)',\
              f'- {num_vert_lines} Vertical ({vert_angles_mean:.2f} +- {vert_angles_std:.2f} deg.)')
        
        #Print update about line selection
        print(f'        ({line_it}): Line Selection: {num_hori_lines_chosen} Horizontal - {num_vert_lines_chosen} Vertical\n')
        
        #Determine if there are at least two lines in each direction after filtration
        sufficient_lines = (num_hori_lines_chosen >= 2) & (num_vert_lines_chosen >= 2)    #looks only at filtered lines
        
        #Determine if the line detection is of a sufficient quality 
        acceptable_lines = True
        
        #Determine if line detection/selection needs to be repeated
        line_it += 1
        if sufficient_lines and acceptable_lines:
            line_repeat = False
        else:
            
            #Update edge detection and line parameters
            if (num_hori_lines < 3) or (num_vert_lines < 3):
                min_line_length = int(0.8 * min_line_length)
                max_line_gap = int(0.33 * min_line_length)
                hough_threshold = int(0.5 * min_line_length)
            elif (num_hori_lines > 100) or (num_vert_lines > 100):
                min_line_length = int(1.1 * min_line_length)
                max_line_gap = int(0.33 * min_line_length)
                hough_threshold = int(0.5 * min_line_length)
            

    #Define perspective correction parameters
    test_std = 0.5
      
        
    #Allocate debug lists
    module_transeds = []
    module_boxes = []
    module_liness = []
    module_edgess = []
    module_transeds.append(module_rgb.copy())   
    module_liness.append(segment.plotLines(module_rgb.copy(), vert_lines, hori_lines, reject_lines))
    module_edgess.append(module_edges)
            
    #Allocate storage lists for paralleity indicators    
    hori_vert_angles = []
    hori_vert_angle_tests = []
    hori_tests = []
    vert_tests = []
    vert_std_tests = []
    hori_std_tests = []
    proj_transs = []
    parallelities = []
        
    #Perspective Correction Selection 
    perspec_it = 0
    correct_perspective = True
    if sufficient_lines:
        
        print('      Perspective Correction Testing:\n')
        for h1 in range(0, num_hori_lines_chosen):
            for h2 in range(h1+1, num_hori_lines_chosen):
                for v1 in range(0, num_vert_lines_chosen):
                    for v2 in range(v1+1, num_vert_lines_chosen):
        
                        #Only go through the logic if the image is not acceptable
                        if correct_perspective:

                            #Define indicies
                            input_hori_inds = [hori_inds[h1], hori_inds[h2]]
                            input_vert_inds = [vert_inds[v1], vert_inds[v2]]            
                            #print(input_vert_inds), print(input_hori_inds)

                            #Determine if lines meet parallelity guidelines
                            hori_vert_angle = np.abs(vert_angles_mean - hori_angles_mean)
                            hori_vert_angle_test = (hori_vert_angle > 88) and (hori_vert_angle < 92)
                            hori_test = (hori_angles_mean >= -1) and (hori_angles_mean <= 1)
                            vert_test = (vert_angles_mean >= 89) and (vert_angles_mean <= 91)
                            vert_std_test = (vert_angles_std < test_std)
                            hori_std_test = (hori_angles_std < test_std)
                            if hori_vert_angle_test and hori_test and vert_test and vert_std_test and hori_std_test:
                                correct_perspective = False
                                
                            #Define and store paralleity
                            if hori_test and vert_test and hori_vert_angle_test:
                                parallelity = vert_angles_std + hori_angles_std
                            else:
                                parallelity = 1000
                            parallelities.append(parallelity)
                            
                            

                            #Print parallelity update 
                            print(f'        ({perspec_it}): Horizontal = {hori_test}; Vertical = {vert_test}; Vert. Parallel = {vert_std_test}; Hori. Parallel = {hori_std_test}; Right Angles = {hori_vert_angle_test}')

                            #Log logical tests
                            hori_vert_angles.append(hori_vert_angle)
                            hori_vert_angle_tests.append(hori_vert_angle_test)
                            hori_tests.append(hori_test)
                            vert_tests.append(vert_test)
                            vert_std_tests.append(vert_std_test)
                            hori_std_tests.append(hori_std_test)

                            #Perspective correction - if the module isn't already acceptably rectangular
                            if correct_perspective:

                                #Iterate through line combinations
                                module_transed, proj_trans, rot_ang, module_lines, module_box = segment.projectiveTransform(module_rgb.copy(), lines,\
                                                                                                                       input_vert_inds, input_hori_inds,\
                                                                                                                       module_gray.copy(), module_rows,\
                                                                                                                       1, hough_theta)
                                #Redefine module_rgb and store proj transform
                                test_module_rgb = module_transed.copy()
                                proj_transs.append(proj_trans)

                                #Log transformed images
                                if debug:
                                    module_boxes.append(module_box)
                                    module_transeds.append(module_rgb)

                                #Detect Edges in the new image 
                                test_module_gray = cv.cvtColor(test_module_rgb.copy(), cv.COLOR_RGB2GRAY)
                                test_module_edges, test_module_edge_outputs = segment.edgeDetection(test_module_rgb.copy(), module_rgb_min, test_module_gray.copy(), module_hue, module_pixels,remove_total_black=True, params=edge_params, debug=False)

                                #Detect Lines 
                                test_lines, test_line_points, test_line_points_edge, test_line_angles = segment.lineDetection(test_module_edges, hough_rho, hough_theta, \
                                                                                          hough_threshold, min_line_length,\
                                                                                          max_line_gap, module_rows, module_cols, \
                                                                                          debug=debug)
                                #Filter lines 
                                test_vert_inds, test_vert_lines, test_vert_angles, test_hori_inds, test_hori_lines, test_hori_angles, test_reject_inds,\
                                test_reject_lines, test_line_outputs = segment.filterLines(test_lines, test_line_points, test_line_angles, hough_theta,\
                                                                                           vert_select_mode='histogram', hori_select_ct=4,\
                                                                                           vert_select_ct=4, debug = True)

                                #Pull line statistics
                                vert_lines_stats = {'angles': test_line_outputs['vert_angles_all'],'angles_mean': test_line_outputs['vert_angles_mean'], \
                                                    'angles_std': test_line_outputs['vert_angles_std']}
                                hori_lines_stats = {'angles': test_line_outputs['hori_angles_all'],'angles_mean': test_line_outputs['hori_angles_mean'], \
                                                    'angles_std': test_line_outputs['hori_angles_std']}

                                #Pull mean angles and stds
                                vert_angles_mean = vert_lines_stats['angles_mean']
                                vert_angles_std = vert_lines_stats['angles_std']
                                hori_angles_mean = hori_lines_stats['angles_mean']
                                hori_angles_std = hori_lines_stats['angles_std']  

                                #Count number of final lines
                                num_lines = len(lines)
                                num_hori_lines = len(hori_lines_stats['angles'])
                                num_vert_lines = len(vert_lines_stats['angles'])
                                num_hori_lines_chosen = len(hori_lines)
                                num_vert_lines_chosen = len(vert_lines)
                                num_lines_filtered = num_lines -(num_hori_lines + num_vert_lines)

                                #Print update about all lines detected (not exclusively filtered lines)
                                print(f'        ({perspec_it}): Line Detection: {num_lines} Detected - '\
                                      f'{num_hori_lines} Horizontal ({hori_angles_mean:.2f} +- {hori_angles_std:.2f} deg.)',\
                                      f'- {num_vert_lines} Vertical ({vert_angles_mean:.2f} +- {vert_angles_std:.2f} deg.)\n')



                                #Update iteration counter
                                perspec_it += 1
                        
            
                                #Log images
                                if debug:
                                    module_edgess.append(test_module_edges)
                                    module_liness.append(segment.plotLines(test_module_rgb.copy(), test_vert_lines, test_hori_lines, test_reject_lines))
        
    else:
        print(f'        -Insufficient lines remain after filtering. Image likely unacceptable.')
        line_repeat = False

    

    #Select best projective transform if none meet initial guidelines
    if correct_perspective and sufficient_lines:
        
        #Select best projective transform (which produces most acceptable lines)
        most_parallel = np.argmin(parallelities)
        module_rgb = module_transeds[most_parallel]
        
    elif (perspec_it > 0):
        module_rgb = test_module_rgb
        
    else:
        module_rgb = module_rgb     #Just stating that it passes through
    
        
        
    
    #Plot lines in image and display
    if debug:
        
        #Define figures 
        perspec_fig = plt.figure(num=f'Perspective Correction Progression', tight_layout=True,figsize=[18, 9.5])
        lines_fig = plt.figure(num=f'Line Detection Progression', tight_layout=True,figsize=[18, 9.5])
        boxes_fig = plt.figure(num=f'Perspective Correction Boxes Progression', tight_layout=True,figsize=[18, 9.5])
        edges_fig = plt.figure(num=f'Edge Detection Progression', tight_layout=True,figsize=[18, 9.5])
        
        #Define subplot params
        sub_rows = 3
        sub_cols = 10
        sub_ct = sub_rows * sub_cols
        
        #Iterate through images and plot the perspective correction refinement images
        for i in range(0, len(module_transeds)):
            
            if i < sub_ct:
                
                #Perspective Correction
                plt.figure('Perspective Correction Progression')
                ax1 = plt.subplot(sub_rows,sub_cols,i+1)
                plt.imshow(module_transeds[i])
                plt.title(f'Iteration {i}')

                #Perspective Correction Boxes
                if i < (len(module_transeds)):
                    plt.figure('Perspective Correction Boxes Progression')
                    ax1 = plt.subplot(sub_rows,sub_cols,i+1)
                    #plt.imshow(module_boxes[i])
                    plt.title(f'Iteration {i}')
            
        for i in range(0, len(module_liness)):
        
            if i < sub_ct:
                
                #Edge Detection
                plt.figure('Edge Detection Progression')
                ax1 = plt.subplot(sub_rows,sub_cols,i+1)
                plt.imshow(module_edgess[i])
                plt.title(f'Iteration {i}')

                #Line Detection
                plt.figure('Line Detection Progression')
                ax1 = plt.subplot(sub_rows,sub_cols,i+1)
                plt.imshow(module_liness[i])
                plt.title(f'Iteration {i}') 
        
        #Change figure
        plt.figure('Module Perspective Correction Debug')
        
        #Plot final edge detection image
        axe = module_fig.add_subplot(gs[0,1])
        plt.imshow(module_edges)
        plt.title('Module Edges')
        output_name = module_root + '_edges' + images_ext
        output_path = os.path.join(debug_dir, output_name)
        #cv.imwrite(output_path, module_edges)
        
        #Plot segments detected
        plt.figure("Module Perspective Correction Debug")
        lines_bgr = segment.plotLines(module_bgr.copy(), vert_lines, hori_lines, reject_lines)
        axls = module_fig.add_subplot(gs[1,0])
        plt.imshow(lines_bgr)
        plt.title('Detected line segments')
        output_name = module_root + '_lines_segments' + images_ext
        output_path = os.path.join(debug_dir, output_name)
        #cv.imwrite(output_path, cv.cvtColor(lines_bgr, cv.COLOR_BGR2RGB))
        
        #Plot the segments extended to the edges
        lines_bgr_extended = segment.plotLines(module_bgr.copy(), None, None, None, all_lines = line_points_edge)
        axlse = module_fig.add_subplot(gs[1,1])
        plt.imshow(lines_bgr_extended)
        plt.title('Extended line segments')
        output_name = module_root + '_lines_extended' + images_ext
        output_path = os.path.join(debug_dir, output_name)
        #cv.imwrite(output_path, cv.cvtColor(lines_bgr_extended, cv.COLOR_BGR2RGB))
        
        #Plot detected total black regions in input image
        axblk = module_fig.add_subplot(gs[1,2])
        plt.imshow(module_total_black)
        plt.title('Total Black Regions')
        
        
    
        #Save all figures 
        output_path = os.path.join(module_debug_output_dir, module_root + '_perspec_debug' + images_ext)
        perspec_fig.savefig(output_path)
        output_path = os.path.join(module_debug_output_dir, module_root + '_perspec_boxes_debug' + images_ext)
        boxes_fig.savefig(output_path)
        output_path = os.path.join(module_debug_output_dir, module_root + '_edge_debug' + images_ext)
        edges_fig.savefig(output_path)
        output_path = os.path.join(module_debug_output_dir, module_root + '_lines_debug' + images_ext)
        lines_fig.savefig(output_path)
        output_path = os.path.join(module_debug_output_dir, module_root + '_line_debug' + images_ext)
        module_fig.savefig(output_path)
        
    
    
    
    #Save final image
    output_name = module_root + '_crop_final' + images_ext
    output_path = os.path.join(debug_dir, output_name)
    cv.imwrite(output_path, module_rgb)
    
    
    
    plt.close('all')
    


## Module Binarization

In [ ]:
#Binarization - Apply Substantial Blur

def moduleBinarization(debug=True):

    #Apply a blur
    gauss_std = 1.5
    module_bgr_blur = cv.GaussianBlur(module_bgr, (25, 25), 1)    
    
    #Binarization - Color space representations
    module_hsv_blur = cv.cvtColor(module_bgr_blur, cv.COLOR_BGR2HSV)
    module_hue_blur = module_hsv_blur[:,:,0]
    module_sat_blur = module_hsv_blur[:,:,1]
    module_val_blur = module_hsv_blur[:,:,2]
    module_blue_blur = module_bgr_blur[:,:,0]
    module_green_blur = module_bgr_blur[:,:,1]
    module_gray_blur = cv.cvtColor(module_bgr_blur, cv.COLOR_BGR2GRAY)
    
    #Binarization - Roseplot and Histogram generation
    hist_blue = cv.calcHist([module_blue_blur], [0], None, [256], [0,256])
    hist_green = cv.calcHist([module_green_blur], [0], None, [256], [0,256])
    hist_gray = cv.calcHist([module_gray_blur], [0], None, [256], [0,256])
    rose_hue = cv.calcHist([module_hue_blur], [0], None, [180], [0,180])
    hist_sat = cv.calcHist([module_sat_blur], [0], None, [256], [0,256])
    hist_val = cv.calcHist([module_val_blur], [0], None, [256], [0,256])
    
    
    #Binarization thresholding - OTSU
    thresh_blue, binary_blue = cv.threshold(module_blue_blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    thresh_green, binary_green = cv.threshold(module_green_blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    thresh_gray, binary_gray = cv.threshold(module_gray_blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    thresh_sat, binary_sat = cv.threshold(module_sat_blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    thresh_val, binary_val = cv.threshold(module_val_blur, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    
    #Binary thresholding - Adaptive
    block_size = 17
    binary_blue_adp = cv.adaptiveThreshold(module_blue_blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, block_size, 2)
    binary_green_adp = cv.adaptiveThreshold(module_green_blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, block_size, 2)
    binary_gray_adp = cv.adaptiveThreshold(module_gray_blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, block_size, 2)
    binary_sat_adp = cv.adaptiveThreshold(module_sat_blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, block_size, 2)
    binary_val_adp = cv.adaptiveThreshold(module_val_blur, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, block_size, 2)
    
    #Binarization - Histogram outputs
    if debug:
        
        #Create figure 
        hist_fig = plt.figure(num=f'Binarization Histogram Debug', tight_layout=True,figsize=[18, 9.5])
        
        #Plot histograms
        ax_gray = plt.subplot(2,3,1)
        plt.plot(hist_gray)
        plt.title('Grayscale Histogram')
        
        ax_blue = plt.subplot(2,3,2)
        plt.plot(hist_blue)
        plt.title('Blue Histogram')
        
        ax_green = plt.subplot(2,3,3)
        plt.plot(hist_green)
        plt.title('Green Histogram')
        
        #ax_hue = plt.subplot(2,3,4, projection='polar')
        #plt.bar(np.linspace(0,360,num=181), rose_hue)
        #plt.title('Grayscale Histogram')
        
        ax_sat = plt.subplot(2,3,5)
        plt.plot(hist_sat)
        plt.title('Saturation Histogram')
        
        ax_val = plt.subplot(2,3,6)
        plt.plot(hist_val)
        plt.title('Value Histogram')
    
    #Binarization - Thresholding outputs
    if debug:
        
        #Create figure
        binary_fig = plt.figure(num=f'Binarization Debug', tight_layout=True,figsize=[18, 9.5])
     
        #Show images
        ax_gray = plt.subplot(3,4,1)
        plt.imshow(binary_gray)
        plt.title('Grayscale OTSU Global')
    
        ax_blue = plt.subplot(3,4,5)
        plt.imshow(binary_blue)
        plt.title('Blue OTSU Global')
        
        ax_green = plt.subplot(3,4,9)
        plt.imshow(binary_green)
        plt.title('Green OTSU Global')
        
        ax_gray = plt.subplot(3,4,2)
        plt.imshow(binary_gray_adp)
        plt.title('Grayscale Gauss Adaptive')
    
        ax_blue = plt.subplot(3,4,6)
        plt.imshow(binary_blue_adp)
        plt.title('Blue Gauss Adaptive')
        
        ax_green = plt.subplot(3,4,10)
        plt.imshow(binary_green_adp)
        plt.title('Green Gauss Adaptive')
        
        ax_sat = plt.subplot(3,4,7)
        plt.imshow(binary_sat)
        plt.title('Saturation OTSU Global')
        
        ax_val = plt.subplot(3,4,11)
        plt.imshow(binary_val)
        plt.title('Value OTSU Global')
    
        ax_sat_adp = plt.subplot(3,4,8)
        plt.imshow(binary_sat_adp)
        plt.title('Saturation Gauss Adaptive')
        
        ax_val_adp = plt.subplot(3,4,12)
        plt.imshow(binary_val_adp)
        plt.title('Value Gauss Adaptive')
        
        
        
    output_path = os.path.join(module_debug_output_dir, module_root + '_binary_debug' + images_ext)
    binary_fig.savefig(output_path)
    output_path = os.path.join(module_debug_output_dir, module_root + '_hist_debug' + images_ext)
    hist_fig.savefig(output_path)

## Frame Outline Detection

In [ ]:
def detectFrameOutline(rgb, debug=True):
    
    #Convert image to other representations
    gray = cv.cvtColor(rgb, cv.COLOR_RGB2GRAY)
    hsv = cv.cvtColor(rgb, cv.COLOR_RGB2HSV)
    h = hsv[:,:,0]
    s = hsv[:,:,1]
    v = hsv[:,:,2]
    r = rgb[:,:,0]
    g = rgb[:,:,1]
    b = rgb[:,:,2]
    
    #Apply gaussian filter
    gray = cv.GaussianBlur(gray, (49, 49), 1)
    
    #Debug display all the color channels
    if debug:
        chan_fig = plt.figure(num=f'Frame Outline Detection Debug - Color Spaces', tight_layout=True,figsize=[18, 9.5])
        plt.subplot(341), plt.imshow(rgb), plt.title('RGB')
        plt.subplot(342), plt.imshow(r), plt.title('R')
        plt.subplot(343), plt.imshow(g), plt.title('G')
        plt.subplot(344), plt.imshow(b), plt.title('B')
        plt.subplot(345), plt.imshow(hsv), plt.title('HSV')
        plt.subplot(346), plt.imshow(h), plt.title('H')
        plt.subplot(347), plt.imshow(s), plt.title('S')
        plt.subplot(348), plt.imshow(v), plt.title('V')
        plt.subplot(349), plt.imshow(gray), plt.title('Grayscale')
        
    
    #Pull image size
    rows, cols, channels = rgb.shape 
    if rows > cols:
        max_dim = rows
        min_dim = cols
        portrait = True
        orientation = 'vertically'
    else:
        max_dim = cols
        min_dim = rows
        portrait = False          
        orientation = 'horizontally'

    #Feedback
    if debug:
        print(f'       Module is oriented {orientation}.')
        plt.figure()
        plt.imshow(rgb)
    
       
    #Define Segment indicies
    max_dim_inds = np.linspace(0, max_dim, 4)   #three segments
    min_dim_inds = np.linspace(0, min_dim, 3)   #two segments

    print(min_dim_inds), print(max_dim_inds)
    
    #Divide segments
    seg_fig = plt.figure(num=f'Frame Outline Detection Debug - Segments', tight_layout=True,figsize=[18, 9.5])
    segments_gray = []
    s = 0
    for imin in range(0,2):
        
        for imax in range(0,3):
            #Define indicies to pull each directions pixel indicies from
            #imin = np.mod(s, 2)
            #imax = np.mod(s, 3)
            #Pull segment from image
            if portrait:  
                x1 = int(min_dim_inds[imin])
                x2 = int(min_dim_inds[imin + 1])
                y1 = int(max_dim_inds[imax])
                y2 = int(max_dim_inds[imax + 1])
                segment_gray = gray[x1:x2, y1:y2]
                segments_gray.append(segment_gray)
            else:
                x1 = int(min_dim_inds[imin])
                x2 = int(min_dim_inds[imin + 1])
                y1 = int(max_dim_inds[imax])
                y2 = int(max_dim_inds[imax + 1])
                segment_gray = gray[x1:x2, y1:y2]
                segments_gray.append(segment_gray)

            #Display segment
            if debug:
                plt.figure('Frame Outline Detection Debug - Segments')
                plt.subplot(2,3,s+1), plt.title(f'Segment {s}')
                plt.imshow(segment_gray)

            #Iterate through rows - calculate statistics
            x_lines_gray = []
            x_der1s_gray = []
            x_der2s_gray = []
            x_means_gray = []
            x_mins_gray = []
            x_maxs_gray = []
            x_stds_gray = []
            x_maxs_der1_gray = []
            x_extrema_diffs_gray = []
            for r in range(0, (x2-x1)):

                #Pull data
                line_gray = segment_gray[r, :]
                der1_gray = np.diff(line_gray)
                der2_gray = np.diff(der1_gray)

                #Calculate statistics
                x_means_gray.append(np.mean(line_gray))
                x_mins_gray.append(np.min(line_gray))
                x_maxs_gray.append(np.max(line_gray))
                x_stds_gray.append(np.std(line_gray))
                x_maxs_der1_gray.append(np.max(der1_gray))
                x_extrema_diffs_gray.append(np.max(line_gray) - np.min(line_gray))

                #Store lines
                x_lines_gray.append(line_gray)
                x_der1s_gray.append(der1_gray)
                x_der2s_gray.append(der2_gray)


            #Calculate row aggregate statistics  
            x_mean_gray = np.mean(x_means_gray)
            x_mean_std_gray = np.std(x_means_gray)
            x_max_gray = np.mean(x_maxs_gray)
            x_max_std_gray = np.std(x_maxs_gray)
            x_extrema_diff_gray = np.mean(x_extrema_diffs_gray)
            x_extrema_diff_std_gray = np.std(x_extrema_diffs_gray)

            #Filter lines into two groups: foreground and background
            x_mean_test = x_means_gray >=(x_mean_gray - 1 * x_mean_std_gray)
            x_max_test = x_maxs_gray >= (x_max_gray - 1 * x_max_std_gray)
            x_extrema_diff_test = x_extrema_diffs_gray >= (x_extrema_diff_gray - 0.5 * x_extrema_diff_std_gray)        
            x_backgrnd_gray = np.where(np.invert(x_mean_test & x_max_test & x_extrema_diff_test))
            x_foregrnd_gray = np.where(x_mean_test & x_max_test & x_extrema_diff_test) 
            x_backgrnd_lines_gray = [x_lines_gray[i] for i in x_backgrnd_gray[0]]
            x_foregrnd_lines_gray = [x_lines_gray[i] for i in x_foregrnd_gray[0]]
            print(x_foregrnd_gray)

            #Iterate through columns - calculate statistics
            y_lines_gray = []
            y_der1s_gray = []
            y_der2s_gray = []
            y_means_gray = []
            y_mins_gray = []
            y_maxs_gray = []
            y_stds_gray = []
            y_maxs_der1_gray = []  
            y_extrema_diffs_gray = []
            for c in range(0, (y2-y1)):

                #Pull data
                line_gray = segment_gray[:, c]
                der1_gray = np.diff(line_gray)
                der2_gray = np.diff(der1_gray)

                #Calculate statistics
                y_means_gray.append(np.mean(line_gray))
                y_mins_gray.append(np.min(line_gray))
                y_maxs_gray.append(np.max(line_gray))
                y_stds_gray.append(np.std(line_gray))
                y_maxs_der1_gray.append(np.max(der1_gray))           
                y_extrema_diffs_gray.append(np.max(line_gray) - np.min(line_gray))

                #Store lines
                y_lines_gray.append(line_gray)
                y_der1s_gray.append(der1_gray)
                y_der2s_gray.append(der2_gray)    


            #Calculate row aggregate statistics  
            y_mean_gray = np.mean(y_means_gray)
            y_mean_std_gray = np.std(y_means_gray)
            y_max_gray = np.mean(y_maxs_gray)
            y_max_std_gray = np.std(y_maxs_gray)
            y_extrema_diff_gray = np.mean(y_extrema_diffs_gray)
            y_extrema_diff_std_gray = np.std(y_extrema_diffs_gray)

            #Filter lines into two groups: foreground and background
            y_mean_test = y_means_gray >=(y_mean_gray - 1 * y_mean_std_gray)
            y_max_test = y_maxs_gray >= (y_max_gray - 1 * y_max_std_gray)
            y_extrema_diff_test = y_extrema_diffs_gray >= (y_extrema_diff_gray - 0.5 * y_extrema_diff_std_gray)        
            y_backgrnd_gray = np.where(np.invert(y_mean_test & y_max_test & y_extrema_diff_test))
            y_foregrnd_gray = np.where(y_mean_test & y_max_test & y_extrema_diff_test) 
            y_backgrnd_lines_gray = [y_lines_gray[i] for i in y_backgrnd_gray[0]]
            y_foregrnd_lines_gray = [y_lines_gray[i] for i in y_foregrnd_gray[0]]
            #print(y_foregrnd_gray)

            

            #Debug figure 
            lines_fig = plt.figure(num=f'Frame Outline Detection Debug - Segment Lines {s}', tight_layout=True,figsize=[18, 9.5])
            plt.subplot(121), plt.title('X Lines')
            for b in range(0, len(x_backgrnd_lines_gray)):
                plt.plot(x_backgrnd_lines_gray[b], 'r-', alpha=0.5)
                
                
            for f in range(0, len(x_foregrnd_lines_gray)):
                plt.plot(x_foregrnd_lines_gray[f], 'g-')
                
                
            plt.subplot(122), plt.title('Y Lines')
            for b in range(0, len(y_backgrnd_lines_gray)):
                plt.plot(y_backgrnd_lines_gray[b], 'r-')
                
                
            for f in range(0, len(y_foregrnd_lines_gray)):
                plt.plot(y_foregrnd_lines_gray[f], 'g-')
                
                
        
            #Iterate segment counter
            s += 1
        
    x=y

## Module Detection with TensorFlow

In [ ]:
#Detection Parameters
probability_thresh = 0.5
broaden_frac = 0.075                  # Scalar for the amount the detection box edges are dilated
shard_images_count = 1
pipeline_type = 'modules'

#Update user
print('Module Detection Setup:')

#Load label map
category_index = label_map_util.create_category_index_from_labelmap(path_to_labels, use_display_name=True)

#Load image pipeline
image_list = os.listdir(prelim_output_dir)
image_list = [image_name for image_name in image_list if image_name.endswith('output'+images_ext)]
image_ct = len(image_list)

#Print update
if image_ct > 0: 
    print(f'   {image_ct} images found in directory meeting image spec.')
else:
    print(f'   No images found in directory meeting spec. Check directories and input spec.')

#Load frozen model into memory
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(path_to_frozen_graph, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
        print(f'   Frozen TensorFlow model loaded.')

graph = detection_graph
with graph.as_default():
    with tf.Session() as sess:
        ops = tf.get_default_graph().get_operations()
        all_tensor_names = {output.name for op in ops for output in op.outputs}
        tensor_dict = {}
        for key in [
            'num_detections', 'detection_boxes', 'detection_scores',
            'detection_classes', 'detection_masks'
        ]:
            tensor_name = key + ':0'
            if tensor_name in all_tensor_names:
                tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                    tensor_name)
        if 'detection_masks' in tensor_dict:
            # The following processing is only for single image
            detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
            detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
            
            # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
            real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
            detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
            detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
            detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                detection_masks, detection_boxes, image.shape[0], image.shape[1])
            detection_masks_reframed = tf.cast(
                tf.greater(detection_masks_reframed, 0.5), tf.uint8)
            
            # Follow the convention by adding back the batch dimension
            tensor_dict['detection_masks'] = tf.expand_dims(
                detection_masks_reframed, 0)
            
        #Define Image Tensor
        image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')   
        print(f'   Model initialized.\n')
        
        
        #TEMPORARY - DEFINE IMAGE SIZE - FUTURE USE SIZE FROM PIPELINE
        image_size = (resize_cols, resize_rows)
        
        #Iterate through images shards
        it = 1
        for image_name in image_list:
            
            #Initialize time tracking
            batch_start_time = time.time()
            
            #User feedback
            print(f'\nProcessing Image {image_name} ({it}):')

            #Load the images
            image_root = os.path.splitext(image_name)[0]
            image_path = os.path.join(prelim_output_dir, image_name)
            image = im.open(image_path).resize(image_size)
            np_image = [detect.load_image_into_numpy_array(image)]
            cv_image = np_image[0].copy()
            
            #Run Detection for Image Set        
            start_time = time.time()
            outputs = detect.run_inference(sess, np_image, detection_graph, tensor_dict, image_tensor)           

            #Count number of modules detected
            num_modules = len(np.where(outputs[0]['detection_scores'] >= probability_thresh)[0])
            print(f'   {num_modules} modules detected.')
            
            #Log Statistics
            #statistics = log_statistics(statistics, probability_thresh, outputs, image_list, np_images)
            
            #Assemble Module
            #np_images, image_list, outputs = assemble_module(np_images, image_list, outputs)
            
            #Generate Visualizations on images
            np_image_labeled = detect.build_visualization(np_image, outputs, category_index, probability_thresh)
            
            #Saving the images
            detect.output_images(np_image_labeled, [image_name], detect_output_dir)
            
            #Calculate module statistics
            module_boxes = [outputs[0]['detection_boxes'][i] for i in range(0,num_modules)]
            module_widths = [module_boxes[i][3] - module_boxes[i][1] for i in range(0,num_modules)]
            module_heights = [module_boxes[i][2] - module_boxes[i][0] for i in range(0,num_modules)]
            module_spacing = [module_boxes[i+1][3] - module_boxes[i][3] for i in range(0,num_modules-1)]
            
            #Crop the image to the box
            crop_images = []
            for m in range(0,num_modules):
                
                #Print update
                print(f'\n   Module {m+1} Perspective Refinement:')
                
                #Pull initial coordinates
                box = module_boxes[m]
                y1 = int(resize_cols * box[1])
                y2 = int(resize_cols * box[3])
                x1 = int(resize_rows * box[0])
                x2 = int(resize_rows * box[2])
                
                #Broaden the box to account for errors
                width = module_widths[m]
                height = module_heights[m]
                x_adj = int(height * broaden_frac * resize_rows)
                y_adj = int(width * broaden_frac * resize_cols)
                y1 -= y_adj
                y2 += y_adj
                x1 -= x_adj
                x2 += x_adj
                if x1 < 0:
                    x1 = 0
                if x2 > (resize_rows - 1):
                    x2 = resize_rows - 1
                if y1 < 0:
                    y1 = 0
                if y2 > (resize_cols - 1):
                    y2 = resize_cols - 1
                    
                #Crop image and size
                crop_image = cv_image[x1:x2, y1:y2, :]    
                crop_rows, crop_cols, crop_channels = crop_image.shape
                crop_pixels = crop_rows * crop_cols
                
                #Store and save cropped image
                crop_images.append(crop_image)
                module_root = image_root + f'-{m}'
                output_name = module_root + images_ext
                output_path = os.path.join(final_output_dir, output_name)
                cv.imwrite(output_path, cv.cvtColor(crop_image, cv.COLOR_BGR2RGB))
                
                #Color space conversion
                crop_gray = cv.cvtColor(crop_image, cv.COLOR_BGR2GRAY)
                
                #Final Perspective Correction
                correctModulePerspective(crop_image, module_root, debug_dir=module_debug_output_dir, debug=debug)
                
                
                #Debug outputs
                if debug:
                    
                    #Create figure
                    fig = plt.figure(num=f'Image {image_name} Preprocessing', tight_layout=True,figsize=[18, 9.5])
                    gs = gridspec.GridSpec(4,5)
                    mng = plt.get_current_fig_manager()
                
                
                
            
          
            
            
            #Feedback
            
            
            #Count which batch
            it += 1

Module Detection Setup:
   341 images found in directory meeting image spec.
   Frozen TensorFlow model loaded.
   Model initialized.


Processing Image DSC03003_output.JPG (1):
   2 modules detected.

   Module 1 Perspective Refinement:
      Image Size is 250 x 424 (106000 px.)

      Line Detection/Selection in Original Image:

        (0): Edge detection: 1 iterations - Ratio = 19.22% - Thresholds = (120, 240)
        (0): Line Detection: 115 Detected - 67 Horizontal (0.10 +- 0.45 deg.) - 12 Vertical (90.65 +- 0.65 deg.)
        (0): Line Selection: 4 Horizontal - 4 Vertical

      Perspective Correction Testing:

        (0): Horizontal = True; Vertical = True; Vert. Parallel = False; Hori. Parallel = True; Right Angles = True
        (0): Line Detection: 115 Detected - 49 Horizontal (0.04 +- 0.36 deg.) - 8 Vertical (90.25 +- 0.37 deg.)

        (1): Horizontal = True; Vertical = True; Vert. Parallel = True; Hori. Parallel = True; Right Angles = True

   Module 2 Perspective Refin

        (29): Line Detection: 113 Detected - 66 Horizontal (-0.01 +- 0.57 deg.) - 12 Vertical (90.27 +- 0.87 deg.)

        (30): Horizontal = True; Vertical = True; Vert. Parallel = False; Hori. Parallel = False; Right Angles = True
        (30): Line Detection: 113 Detected - 66 Horizontal (-1.45 +- 1.48 deg.) - 6 Vertical (90.15 +- 0.98 deg.)

        (31): Horizontal = False; Vertical = True; Vert. Parallel = False; Hori. Parallel = False; Right Angles = True
        (31): Line Detection: 113 Detected - 63 Horizontal (-1.42 +- 1.26 deg.) - 6 Vertical (90.27 +- 0.37 deg.)

        (32): Horizontal = False; Vertical = True; Vert. Parallel = True; Hori. Parallel = False; Right Angles = True
        (32): Line Detection: 113 Detected - 49 Horizontal (-1.60 +- 0.88 deg.) - 8 Vertical (90.37 +- 0.68 deg.)

        (33): Horizontal = False; Vertical = True; Vert. Parallel = False; Hori. Parallel = False; Right Angles = True
        (33): Line Detection: 113 Detected - 60 Horizontal (-1.58

        (8): Line Detection: 96 Detected - 43 Horizontal (0.66 +- 0.67 deg.) - 6 Vertical (90.06 +- 0.71 deg.)

        (9): Horizontal = True; Vertical = True; Vert. Parallel = False; Hori. Parallel = False; Right Angles = True
        (9): Line Detection: 96 Detected - 40 Horizontal (0.64 +- 0.65 deg.) - 4 Vertical (89.87 +- 0.30 deg.)

        (10): Horizontal = True; Vertical = True; Vert. Parallel = True; Hori. Parallel = False; Right Angles = True
        (10): Line Detection: 96 Detected - 43 Horizontal (0.60 +- 0.71 deg.) - 9 Vertical (89.78 +- 0.59 deg.)

        (11): Horizontal = True; Vertical = True; Vert. Parallel = False; Hori. Parallel = False; Right Angles = True
        (11): Line Detection: 96 Detected - 44 Horizontal (0.58 +- 0.66 deg.) - 10 Vertical (89.91 +- 0.70 deg.)

        (12): Horizontal = True; Vertical = True; Vert. Parallel = False; Hori. Parallel = False; Right Angles = True
        (12): Line Detection: 96 Detected - 40 Horizontal (0.74 +- 0.65 deg.) -


Processing Image DSC03024_output.JPG (6):
   2 modules detected.

   Module 1 Perspective Refinement:
      Image Size is 248 x 410 (101680 px.)

      Line Detection/Selection in Original Image:

        (0): Edge detection: 3 iterations - Ratio = 19.93% - Thresholds = (102, 204)
        (0): Line Detection: 96 Detected - 41 Horizontal (-0.06 +- 0.15 deg.) - 6 Vertical (90.08 +- 0.42 deg.)
        (0): Line Selection: 4 Horizontal - 4 Vertical

      Perspective Correction Testing:

        (0): Horizontal = True; Vertical = True; Vert. Parallel = True; Hori. Parallel = True; Right Angles = True

   Module 2 Perspective Refinement:
      Image Size is 242 x 404 (97768 px.)

      Line Detection/Selection in Original Image:

        (0): Edge detection: 10 iterations - Ratio = 19.57% - Thresholds = (134, 269)
        (0): Line Detection: 104 Detected - 56 Horizontal (-0.04 +- 0.32 deg.) - 10 Vertical (89.80 +- 0.56 deg.)
        (0): Line Selection: 4 Horizontal - 2 Vertical

      Pe